内存分配器：
- 以某个阈值为界，分⼤小对象
- 大对象直接分配内存，只小对象才使⽤用专⽤用内存分配器器
- 在通⽤用内存分配基础上，以缓存⽅方式实现小对象快速分配

python3.6中，大小对象以512字节为界，Arena为512KB，Pool为4KB。

|Request in bytes |Size of allocated block |Size class idx|
|-----------------|-------------------------|----------------------| 
|1-8 |8 |0|
|9-16 |16 |1|
|17-24 |24 |2|
|25-32 |32 |3|
|... |... |...|
|497-504 |504 |62|
|505-512 |512 |63|

小对象按8字节对齐，分成64个不同的类别。

- 向系统申请⼤块 arena 内存
- 分成多个 pool 块(⼀级重⽤用单元)，为⼀种类别提供内存
- 按类别，分成多个 block 块(⼆级)。可存储相同类别的⼩对象